In [37]:
from dbfread import DBF
import pandas as pd
from simpledbf import Dbf5
from datetime import datetime
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import  cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
def remover(df, colunas):
    return df.drop(columns=colunas, errors='ignore')

In [3]:
def binario(colunaA):
    empty = []
    colunaA = colunaA.to_list()  
    for valor in colunaA:
        if valor in [1, 2]:  
            empty.append(1)
        else:
            empty.append(0)
    return empty

In [4]:
def extrair(colunaA):
    empty = []
    
    for stra in colunaA:
        if pd.isna(stra): 
            empty.append(np.nan)  
            continue
        try:
            stra_value = stra.split(" ")[1]  
            empty.append(stra_value)  
        except IndexError:
            empty.append(np.nan)  
        except Exception as e:
            print(f"Error processing entry '{stra}': {e}")
            empty.append(np.nan)

    return empty

In [5]:
def diferencaA(colunaA, colunaB):
    empty = []
    colunaA = colunaA.to_list()
    colunaB = colunaB.to_list()
    for i in range(len(colunaA)):
        date_format = "%Y-%m-%d"
        coluna_a = datetime.strptime(colunaA[i], date_format)
        coluna_b = datetime.strptime(colunaB[i], date_format)
        dif = (coluna_b - coluna_a).days
        if dif <= 30 :
            empty.append(0)
        elif dif <= 60 :
            empty.append(1)
        else:
            empty.append(2)
    return empty

In [6]:
def diferencaB(colunaA, colunaB):

    colunaA = colunaA.to_list()
    colunaB = colunaB.to_list()
    
    empty = []
    date_format = "%Y-%m-%d" 
    
    for date_a, date_b in zip(colunaA, colunaB):
        if pd.isna(date_a) or pd.isna(date_b):
            empty.append('Não tratou')
            continue

        coluna_a = datetime.strptime(date_a, date_format)
        coluna_b = datetime.strptime(date_b, date_format)
        dif = (coluna_b - coluna_a).days

        if dif <= 30:
            empty.append(0)
        elif dif <= 60:
            empty.append(1)
        elif dif <= 90:
            empty.append(2)
        else:
            empty.append('Não tratou')

    return empty

In [7]:
dbfile = Dbf5('pacigeral_jun24.dbf', codec='latin-1')
dbfile.to_csv("paciente_geral_v2.csv")
pd.set_option('display.max_columns', None)


In [8]:
df = pd.read_csv('paciente_geral_v2.csv', encoding='latin-1', sep=',')

C:\Users\silva\AppData\Local\Temp\ipykernel_8244\4039390663.py:1: DtypeWarning: Columns (37,38,39,40,77,78,79,82,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('paciente_geral_v2.csv', encoding='latin-1', sep=',')


In [9]:
# 1)
df_ajustado = df[df['TOPOGRUP'] == 'C34']
df_ajustado.shape


(56800, 105)

In [10]:
# 2)
df_ajustado = df_ajustado[df_ajustado['UFRESID'] == 'SP']
df_ajustado.shape

(52205, 105)

In [11]:
# 3)
df_ajustado = df_ajustado[df_ajustado['BASEDIAG'] == 3] 
df_ajustado.shape

(50583, 105)

In [12]:
# 4)
df_ajustado = df_ajustado[(df_ajustado['ECGRUP'] != 0) & (df_ajustado['ECGRUP'] != 'X') & (df_ajustado['ECGRUP'] != 'Y')]
df_ajustado.shape

(45452, 105)

In [13]:
# 5)
df_ajustado = df_ajustado[(df_ajustado['HORMONIO'] != 1) & (df_ajustado['TMO'] != 1)]
df_ajustado.shape

(45241, 105)

In [14]:
# 6)
df_ajustado = df_ajustado[df_ajustado['ANODIAG'] <= 2019]
df_ajustado.shape

(38771, 105)

In [15]:
# 7)
df_ajustado = df_ajustado[df_ajustado['IDADE'] >= 20]
df_ajustado.shape


(38759, 105)

In [16]:
# 8.1)
df_ajustado['CONSDIAG'] = diferencaA(df_ajustado['DTCONSULT'], df_ajustado['DTDIAG'])
df_ajustado.shape

(38759, 105)

In [17]:
# 8.2)
df_ajustado['DIAGTRAT'] = diferencaB(df_ajustado['DTDIAG'], df_ajustado['DTTRAT'])
df_ajustado.shape

(38759, 105)

In [18]:
# 8.3)
df_ajustado['TRATCONS'] = diferencaB(df_ajustado['DTCONSULT'], df_ajustado['DTTRAT'])
df_ajustado.shape

(38759, 105)

In [19]:
# 9.1)
df_ajustado['DRS'] = extrair(df_ajustado['DRS'])
df_ajustado.shape

(38759, 105)

In [20]:
# 9.2)
df_ajustado['DRSINST'] = extrair(df_ajustado['DRSINST'])
df_ajustado.shape

(38759, 105)

In [21]:
# 10)
df_ajustado['OBITO'] = binario(df_ajustado['ULTINFO'])
df_ajustado.shape

(38759, 106)

In [22]:
# 11)
df_ajustado = remover(df_ajustado,["UFNASC", "UFRESID", "CIDADE", "DTCONSULT", "CLINICA", "DTDIAG",
"BASEDIAG", "TOPOGRUP", "DESCTOPO", "DESCMORFO", "T", "N",
"M", "PT", "PN", "PM", "S", "G", "LOCALTNM", "IDMITOTIC",
"PSA", "GLEASON", "OUTRACLA", "META01", "META02", "META03",
"META04", "DTTRAT", "NAOTRAT", "TRATAMENTO", "TRATHOSP",
"TRATFANTES", "TRATFAPOS", "HORMONIO", "TMO", "NENHUMANT",
"CIRURANT", "RADIOANT", "QUIMIOANT", "HORMOANT", "TMOANT",
"IMUNOANT", "OUTROANT", "HORMOAPOS", "TMOAPOS", "DTULTINFO",
"CICI", "CICIGRUP", "CICISUBGRU", "FAIXAETAR", "LATERALI",
"INSTORIG", "RRAS", "ERRO", "DTRECIDIVA", "RECNENHUM",
"RECLOCAL", "RECREGIO", "RECDIST", "REC01", "REC02",
"REC03", "REC04", "CID", "DSCCIDO", "HABILIT", "HABIT11",
"HABILIT1", "CIDADEH", "PERDASEG"])
df_ajustado.shape

(38759, 38)

In [23]:
df_ajustado.head()

,INSTITU,DSCINST,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,MORFO,CIDO,DESCIDO,EC,ECGRUP,NENHUM,CIRURGIA,RADIO,QUIMIO,IMUNO,OUTROS,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,DRS,IBGEATEN,DRSINST,RRASINST,CIDADEINST,HABILIT2,OBITO
169,20230,SANTA CASA DE ARARAS,9,73,1,3503307,9,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IB,I,0,1,0,1,0,0,1,0,0,0,0,0,3,0,0,0,2002,10,3503307,10,RRAS 14,ARARAS,1,0
171,20621,SANTA CASA DE ARARAQUARA,9,67,2,3529302,9,1,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IIIB,III,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,2004,03,3503208,03,RRAS 18,ARARAQUARA,1,1
172,18058,HOSP ANCHIETA FUNDACAO ABC,2,72,2,3548708,2,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,1,0,0,0,1,0,0,0,0,0,3,0,1,2,2016,01,3548708,01,RRAS 01,SAO BERNARDO DO CAMPO,1,0
173,19976,HOSP MAT CELSO PIERRO PUCC,9,65,2,3501608,9,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IB,I,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,2010,07,3509502,07,RRAS 15,CAMPINAS,1,1
174,18058,HOSP ANCHIETA FUNDACAO ABC,3,63,1,3548708,2,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,0,1,0,1,1,0,0,0,0,0,3,0,0,2,2015,01,3548708,01,RRAS 01,SAO BERNARDO DO CAMPO,1,0


In [24]:
df_ajustado.info()
#Colunas com variavéis categóricas devem ser tranformadas em numéricas, existem 9 colunas categóricas

<class 'pandas.core.frame.DataFrame'>
Index: 38759 entries, 169 to 1209003
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   INSTITU     38759 non-null  int64  
 1   DSCINST     38759 non-null  object 
 2   ESCOLARI    38759 non-null  int64  
 3   IDADE       38759 non-null  int64  
 4   SEXO        38759 non-null  int64  
 5   IBGE        38759 non-null  int64  
 6   CATEATEND   38759 non-null  int64  
 7   DIAGPREV    38759 non-null  int64  
 8   TOPO        38759 non-null  object 
 9   MORFO       38759 non-null  int64  
 10  CIDO        38759 non-null  float64
 11  DESCIDO     38759 non-null  object 
 12  EC          38759 non-null  object 
 13  ECGRUP      38759 non-null  object 
 14  NENHUM      38759 non-null  int64  
 15  CIRURGIA    38759 non-null  int64  
 16  RADIO       38759 non-null  int64  
 17  QUIMIO      38759 non-null  int64  
 18  IMUNO       38759 non-null  int64  
 19  OUTROS      38759 non-null

In [25]:
df_ajustado.isnull().sum()
#Não existem colunas com Nans, considerarei isto para o pré-processamento

INSTITU       0
DSCINST       0
ESCOLARI      0
IDADE         0
SEXO          0
IBGE          0
CATEATEND     0
DIAGPREV      0
TOPO          0
MORFO         0
CIDO          0
DESCIDO       0
EC            0
ECGRUP        0
NENHUM        0
CIRURGIA      0
RADIO         0
QUIMIO        0
IMUNO         0
OUTROS        0
NENHUMAPOS    0
CIRURAPOS     0
RADIOAPOS     0
QUIMIOAPOS    0
IMUNOAPOS     0
OUTROAPOS     0
ULTINFO       0
CONSDIAG      0
TRATCONS      0
DIAGTRAT      0
ANODIAG       0
DRS           0
IBGEATEN      0
DRSINST       0
RRASINST      0
CIDADEINST    0
HABILIT2      0
OBITO         0
dtype: int64

In [26]:
outliers_summary = {}
for column in df_ajustado.select_dtypes(include=['float64', 'int64']):
    Q1 = df_ajustado[column].quantile(0.25)
    Q3 = df_ajustado[column].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    num_outliers = df_ajustado[(df_ajustado[column] < limite_inferior) | (df_ajustado[column] > limite_superior)].shape[0]
    outliers_summary[column] = num_outliers
print("Número de outliers por coluna:")
for column, count in outliers_summary.items():
    print(f"{column}: {count} outliers")
#Analisdando os outliers porém se concluiu que apenas IDADE há outliers o resto está desbalanceado

Número de outliers por coluna:
INSTITU: 9843 outliers
ESCOLARI: 0 outliers
IDADE: 358 outliers
SEXO: 0 outliers
IBGE: 0 outliers
CATEATEND: 0 outliers
DIAGPREV: 0 outliers
MORFO: 1325 outliers
CIDO: 1325 outliers
NENHUM: 6639 outliers
CIRURGIA: 7381 outliers
RADIO: 0 outliers
QUIMIO: 0 outliers
IMUNO: 174 outliers
OUTROS: 2424 outliers
NENHUMAPOS: 1739 outliers
CIRURAPOS: 74 outliers
RADIOAPOS: 730 outliers
QUIMIOAPOS: 323 outliers
IMUNOAPOS: 12 outliers
OUTROAPOS: 211 outliers
ULTINFO: 7579 outliers
CONSDIAG: 6061 outliers
ANODIAG: 0 outliers
IBGEATEN: 0 outliers
HABILIT2: 2102 outliers
OBITO: 3792 outliers


In [27]:
Q1 = df_ajustado['IDADE'].quantile(0.25)  
Q3 = df_ajustado['IDADE'].quantile(0.75)  
IQR = Q3 - Q1  
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
media = df_ajustado['IDADE'].mean()
df_ajustado['IDADE'] = df_ajustado['IDADE'].apply(lambda x: media if x < limite_inferior or x > limite_superior else x)
#Retirando os outliers e substituindo pela média

In [28]:
df_ajustado['TRATCONS'] = pd.to_numeric(df_ajustado['TRATCONS'], errors='coerce')
median_value = df_ajustado['TRATCONS'].median()
df_ajustado.loc[df_ajustado['TRATCONS'].isna(), 'TRATCONS'] = median_value
df_ajustado['DIAGTRAT'] = pd.to_numeric(df_ajustado['DIAGTRAT'], errors='coerce')
median_value = df_ajustado['DIAGTRAT'].median()
df_ajustado.loc[df_ajustado['DIAGTRAT'].isna(), 'DIAGTRAT'] = median_value
#substituindo os locais com "Não tratou" pela média e criando assim uma coluna numérica

In [29]:
df_ajustado['DRS'] = df_ajustado['DRS'].astype(np.int64)
df_ajustado['DRSINST'] = df_ajustado['DRSINST'].astype(np.int64)
#tranformando as colunas em numéricas

In [30]:
label_encoder = LabelEncoder()
df_ajustado['DSCINST'] = label_encoder.fit_transform(df_ajustado['DSCINST'])
df_ajustado['TOPO'] = label_encoder.fit_transform(df_ajustado['TOPO'])
df_ajustado['DESCIDO'] = label_encoder.fit_transform(df_ajustado['DESCIDO'])
df_ajustado['EC'] = label_encoder.fit_transform(df_ajustado['EC'])
df_ajustado['ECGRUP'] = label_encoder.fit_transform(df_ajustado['ECGRUP'])
df_ajustado['RRASINST'] = label_encoder.fit_transform(df_ajustado['RRASINST'])
df_ajustado['CIDADEINST'] = label_encoder.fit_transform(df_ajustado['CIDADEINST'])
#transformando as colunaas em numéricas com a técnica Label Enconding

In [31]:
df_ajustado.info()
#Todas as categorias foram passadas para numéricas

<class 'pandas.core.frame.DataFrame'>
Index: 38759 entries, 169 to 1209003
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   INSTITU     38759 non-null  int64  
 1   DSCINST     38759 non-null  int64  
 2   ESCOLARI    38759 non-null  int64  
 3   IDADE       38759 non-null  float64
 4   SEXO        38759 non-null  int64  
 5   IBGE        38759 non-null  int64  
 6   CATEATEND   38759 non-null  int64  
 7   DIAGPREV    38759 non-null  int64  
 8   TOPO        38759 non-null  int64  
 9   MORFO       38759 non-null  int64  
 10  CIDO        38759 non-null  float64
 11  DESCIDO     38759 non-null  int64  
 12  EC          38759 non-null  int64  
 13  ECGRUP      38759 non-null  int64  
 14  NENHUM      38759 non-null  int64  
 15  CIRURGIA    38759 non-null  int64  
 16  RADIO       38759 non-null  int64  
 17  QUIMIO      38759 non-null  int64  
 18  IMUNO       38759 non-null  int64  
 19  OUTROS      38759 non-null

In [32]:
df_ajustado = df_ajustado.drop_duplicates()
#retirando colunas duplicadas que podem gerar ruído no treinamento

In [33]:
X = df_ajustado.drop(columns=['OBITO'])  
y = df_ajustado['OBITO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
#Separação da coluna obito e divisão de dados em treinamento e teste, além da implementação do SMOTE para ajudar com o desbalanceamento nas colunas


In [34]:
df_ajustado.shape

(38737, 38)

In [38]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
cv_scores = cross_val_score(rf_model, X_train_bal, y_train_bal, cv=5, scoring='f1')
rf_model.fit(X_train_bal, y_train_bal)
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7019
           1       1.00      1.00      1.00       729

    accuracy                           1.00      7748
   macro avg       1.00      1.00      1.00      7748
weighted avg       1.00      1.00      1.00      7748



Considerações finais:
-Graças a muitas variavéis desbalanceadas, foi gerado um overfitting. Para isso poderíamos utilizar um Undersampling e outros métodos.
-Para melhor performance os códigos da primeira parte do desafio poderiam ter sido feitos de outra forma, que não o append.
-Uma implementação de engineering features, poderia ser satisfatória para o resultado.
-Com tudo os interesses para o desafio foram atendidos.
